# 1. Install Dependencies

In [1]:
!python -m pip install --upgrade pip setuptools wheel
%pip install libtorrent lbry-libtorrent dropbox tomlkit

!curl https://rclone.org/install.sh | sudo bash
%pip install rclone-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 68.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.1/572.1 kB 21.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [dropbox]
  % Total    

# 2. Setup
## 2.1 Create filesystem

In [230]:
import sys
import os, subprocess, glob
from concurrent.futures import ThreadPoolExecutor, Future, as_completed

import logging
import time
from timeit import default_timer as timer

from rich.progress import Progress, TextColumn, BarColumn, TaskProgressColumn, TimeRemainingColumn
from pathlib import Path

from tomlkit import document, table, nl, comment
from tomlkit import dumps
from tomlkit.toml_file import TOMLFile

from configparser import ConfigParser
from pathlib import Path

import libtorrent as lt
from rclone_python import rclone

from google.colab import drive
drive.mount('/content/drive')
import ipywidgets as widgets

# generate file to add
magnet_path = Path("/content/magnet.links")
magnet_file = TOMLFile(magnet_path.as_posix())

if not magnet_path.exists():
  magnet_document = document()

  example_folder_tor = table()
  example_folder_tor["category"] = "tv_shows"
  example_folder_tor["magnets"] = ["mag1"]

  example_movie_tor = table()
  example_movie_tor["category"] = "movies"
  example_movie_tor["magnets"] = ["mag2"]

  example_multifiles = table()
  example_multifiles["category"] = "anime"
  example_multifiles["magnets"] = ["mag3", "mag4", "mag5", "..."]

  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("#  This file explicitly outlines how torrents should be stored.                #"))
  magnet_document.add(comment("################################################################################"))
  magnet_document.add(comment("the category should match the containing folder name in jellyfin. for example: category=/your/path/to/jellyfinmedia/[category]"))
  magnet_document.add("awesome_tv_show", example_folder_tor)
  magnet_document.add("exciting_movie", example_movie_tor)
  magnet_document.add(nl())
  magnet_document.add(comment("seperate file episodes"))
  magnet_document.add("cool_anime", example_multifiles)

  magnet_file.write(magnet_document)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Configure app

In [231]:
# observed that google has this file and copy that config file
gdrive_rclone_savepath = Path("/content/drive/MyDrive/Apps/jellyfin-tor/rclone/rclone.conf")
rclone_path = Path("/root/.config/rclone/rclone.conf")

# check if the config file exists in google drive
if gdrive_rclone_savepath.exists():
  rclone_path.parent.mkdir(parents=True, exist_ok=True)
  !cp {gdrive_savepath.as_posix()} {rclone_path.as_posix()}

# first time setup if the config file doesn't exist in the cloud
if not rclone_path.exists():
  rclone_path.parent.mkdir(parents=True, exist_ok=True)
  rclone_path.touch()

  # generate the rclone config
  rclone_cfg = ConfigParser()
  rclone_cfg['dropbox'] = \
  {
      "type": "dropbox",
      "token": ""
  }

  # get refresh token from user host
  token = input(
  """
  With rclone installed on a local system outside of google colab,
  run \"rclone authorize dropbox\" and paste the result here:
  """)
  rclone_cfg['dropbox']['token'] = token

  # create the rclone config file
  with rclone_path.open('w') as cfgwriter:
    rclone_cfg.write(cfgwriter)

In [238]:
# get app config
app_conf_path = Path("/content/drive/MyDrive/Apps/jellyfin-tor/jellyfin-tor.conf")

app_cfg = ConfigParser()
if not app_conf_path.exists():
  app_conf_path.parent.mkdir(parents=True, exist_ok=True)
  app_conf_path.touch()

  app_cfg['Jellyfin'] = \
  {
      "media_folder": "",
  }

  with app_conf_path.open('w') as cfgwriter:
    app_cfg.write(cfgwriter)

else:
  app_cfg.read(app_conf_path.as_posix())

# 3. Config app

In [211]:
# @title
# --- Row template as a class ---
class LinkWidget:
    def __init__(self, parent_box):
      self.parent_box = parent_box

      self.btn_up = widgets.Button(description='↑', button_style='info', layout=widgets.Layout(width="30px"))
      self.btn_down = widgets.Button(description='↓', button_style='info', layout=widgets.Layout(width="30px"))
      self.text = widgets.Text(placeholder='magnet:?xt=urn:btih:', layout=widgets.Layout(width="50%"))
      self.btn_add = widgets.Button(description='+', button_style='success', layout=widgets.Layout(width="30px"))
      self.btn_remove = widgets.Button(description='-', button_style='danger', layout=widgets.Layout(width="30px"))

      self.widget = widgets.HBox([self.btn_up, self.btn_down, self.text, self.btn_add, self.btn_remove])

      # wire button actions
      self.btn_add.on_click(self.add_row)
      self.btn_remove.on_click(self.remove_row)
      self.btn_up.on_click(self.move_up)
      self.btn_down.on_click(self.move_down)

      self.links = list()

    # --- button actions ---
    def add_row(self, _):
      rows = list(self.parent_box.children)
      idx = rows.index(self.widget)
      new_row = LinkWidget(self.parent_box).widget

      # insert right after this row
      rows.insert(idx + 1, new_row)

      self.parent_box.children = rows


    def remove_row(self, _):
      new_children = list(self.parent_box.children)
      if(len(new_children)) <= 1:
        return
      new_children.remove(self.widget)
      self.parent_box.children = new_children

    def move_up(self, _):
      rows = list(self.parent_box.children)
      idx = rows.index(self.widget)
      if idx > 0:
          rows[idx], rows[idx - 1] = rows[idx - 1], rows[idx]
          self.parent_box.children = rows

    def move_down(self, _):
      rows = list(self.parent_box.children)
      idx = rows.index(self.widget)
      if idx < len(rows) - 1:
          rows[idx], rows[idx + 1] = rows[idx + 1], rows[idx]
          self.parent_box.children = rows

    def obtain_link(self):
      self.links.clear()
      for hbox in self.parent_box.children:
        link = hbox.children[2].value
        if link == "":
          continue

        self.links.append(hbox.children[2].value)

In [182]:
# @title
class MediaModule:
  def __init__(self, manager):
    self.manager = manager

    self.title_textbox = widgets.Text(description="Title", placeholder="Awesome Movie Title", layout=widgets.Layout(width="30%"))
    self.category_textbox = widgets.Text(description="Category", placeholder="movies, tv_shows, anime, ...", layout=widgets.Layout(width="30%"))

    self.link_list = widgets.VBox([])
    self.initial_row = LinkWidget(self.link_list)
    self.link_list.children = [self.initial_row.widget]

    # --- NEW remove button ---
    self.btn_remove = widgets.Button(description="Remove Series", button_style='danger')
    self.btn_remove.on_click(self.remove)

    self.link_list = widgets.VBox([])
    self.initial_row = LinkWidget(self.link_list)
    self.link_list.children = [self.initial_row.widget]

    # place remove button at top or bottom
    self.widget = widgets.VBox([
      self.title_textbox,
      self.category_textbox,
      self.link_list,
      widgets.HTML("<br>"),
      self.btn_remove,
    ])

  def remove(self, _):
    self.manager.remove_module(self)

In [215]:
# @title
class MediaModuleManager:
  def __init__(self):
    self.accordion = widgets.Accordion(children=[])
    self.modules = []

    # Add first module
    self.add_module()

    # Button to add more
    self.add_button = widgets.Button(
        description="Add Module",
        icon="plus",
        button_style="info",
        layout=widgets.Layout(width="20%")
    )
    self.add_button.on_click(self.add_module)

    self.submit_button = widgets.Button(
        description="Submit",
        button_style="success",
        icon='check',
        layout=widgets.Layout(width="20%")
    )
    self.submit_button.on_click(self.submit)

    self.widget = widgets.VBox([self.accordion, self.add_button, self.submit_button])

  def add_module(self, _=None):
    module = MediaModule(self)
    self.modules.append(module)  # append first

    # insert module widget into accordion
    children = list(self.accordion.children)
    children.append(module.widget)
    self.accordion.children = children

    # determine this module’s actual index
    module_index = len(self.modules) - 1

    # set initial title
    self.accordion.set_title(module_index, module.title_textbox.value or f"Series {module_index + 1}")

    # observer that dynamically looks up the module's current index
    def update_title(change):
        idx = self.modules.index(module)
        self.accordion.set_title(idx, change['new'])

    module.title_textbox.observe(update_title, names='value')

    # auto-open newest section
    self.accordion.selected_index = module_index

  def submit(self, _):
    hasError = False
    magnet_document = document()

    # print("=== Submitted Data ===")
    for i, module in enumerate(self.modules):
      elements = table()

      # fetch title + category
      title = module.title_textbox.value
      category = module.category_textbox.value

      if not title:
        hasError = True
        print(f"{i}: Missing Title")

      if not category:
        hasError = True
        print(f"{i}: Missing Category")

      if hasError:
        continue

      elements["category"] = category

      # fetch links
      links = []
      for row in module.link_list.children:
        link = row.children[2].value
        if link.strip():
            links.append(link)

        elements["magnets"] = links
      magnet_document.add(title, elements)
        # print(f"\nModule {i+1}")
        # print("Title:", title)
        # print("Category:", category)
        # print("Links:")
        # for link in links:
        #     print(" •", link)
    magnet_file.write(magnet_document)
    print("=== Successfully wrote to magnet.links ===")

  def remove_module(self, module):
      if len(self.modules) == 1:
          return  # do not remove last module

      # Remove module from list
      self.modules.remove(module)

      # Rebuild accordion children
      self.accordion.children = [m.widget for m in self.modules]

      # Update accordion titles based on current module order
      for i, m in enumerate(self.modules):
          self.accordion.set_title(i, m.title_textbox.value or f"Series {i+1}")



In [228]:
# UI to handle adding media to torrent
manager = MediaModuleManager()
display(manager.widget)

=== Successfully wrote to magnet.links ===


# Download torrent

In [229]:
# TODO: Handle torrent files
def download_torrent(progress: Progress, category:str, media_name: str, link: str):
  MAX_FILENAME_LEN = 35     # rich progress constant
  METADATA_TIMEOUT_MS = .8

  # generate save path for torrent download
  save_path=Path("/content/Torrents/") / category / media_name

  if not save_path.exists():
    save_path.mkdir(parents=True)

  # setup rich progress
  task = progress.add_task(
        "download",
        total=100,
        status="[yellow]Obtaining Metadata",
        filename=media_name,
        speed="0.0 kB/s",
  )
  ses = lt.session()

  # setup torrent handle
  try:
    atp = lt.parse_magnet_uri(link)           # atp = add torrent parameters
  except Exception as e:
    progress.update(task, status="[red]Failed to parse magnet!")
    return None
  atp.save_path = str(save_path.as_posix())

  # generate the metadata to download torrents
  handle = ses.add_torrent(atp)
  status = handle.status()

  while not handle.status().has_metadata:
    time.sleep(METADATA_TIMEOUT_MS/1000)
    status = handle.status()

  filename =  status.name if len(status.name) < MAX_FILENAME_LEN \
              else f"{status.name[:MAX_FILENAME_LEN]}..."

  # start progress bar
  progress.update(task,
                  status="[yellow]Downloading",
                  filename=filename
  )

  # proceed to download the torrent
  while not status.is_seeding:
    status = handle.status()

    progress.update(task,
                completed=status.progress * 100,
                speed=f"{status.download_rate/1000:.1f} kB/s",
    )

  # finished status
  progress.update(task,
                  status="[green]Complete",
                  completed=100,
  )

  return save_path

In [232]:
with Progress(
  TextColumn("{task.fields[status]}"),
  TextColumn("[bold]{task.fields[filename]}"),
  BarColumn(),
  TaskProgressColumn(),
  TimeRemainingColumn(),
  TextColumn("{task.fields[speed]}"),
) as progress:

  # multithread thread downloading
  with ThreadPoolExecutor() as executor:
    media = magnet_file.read()
    futures = []
    for m in media:
      magnet_links = media[m]["magnets"]
      category = media[m]["category"]

      for link in magnet_links:
        futures.append(executor.submit(download_torrent, progress, category, m, link))

    for future in as_completed(futures):
      try:
          future.result()
      except Exception as e:
          print("Upload failed:", e)

Output()

In [241]:
dropbox_filepath = "dropbox:/JellyfinMedia/" # @param {type:"string"}
# rclone.copy('./Torrents/', dropbox_filepath, ignore_existing=True, args=['--create-empty-src-dirs'])

app_cfg['Jellyfin']['media_folder'] = dropbox_filepath
with(app_conf_path.open('w')) as cfgwriter:
  app_cfg.write(cfgwriter)

# save config to google drive for future use
if not gdrive_rclone_savepath.exists():
  gdrive_rclone_savepath.parent.mkdir(parents=True, exist_ok=True)
  gdrive_rclone_savepath.touch()

!cp {rclone_path.as_posix()} {gdrive_rclone_savepath.as_posix()}